# **Importing Library For ML**

In [ ]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from google.colab import files
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

## Importing Dataset From Kaggle

### Mount gdrive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Upload kaggle api key

In [ ]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rizkyadityaichwanto","key":"9e8cda76e29af00972c820904e58788e"}'}

In [ ]:
!ls -lha kaggle.json

-rw-r--r-- 1 root root 75 May 28 13:19 kaggle.json


In [ ]:
!pip install -q kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

Check dir

In [ ]:
!pwd

/content


Download Datasets & Unzipping

In [ ]:
!kaggle datasets download -d frabbisw/facial-age

100% 840M/840M [00:32<00:00, 29.8MB/s]
100% 840M/840M [00:32<00:00, 26.9MB/s]


In [ ]:
!unzip facial-age.zip

Streaming output truncated to the last 5000 lines.
  inflating: face_age/face_age/024/7190.png  
  inflating: face_age/face_age/024/7213.png  
  inflating: face_age/face_age/024/7342.png  
  inflating: face_age/face_age/024/7366.png  
  inflating: face_age/face_age/024/7406.png  
  inflating: face_age/face_age/024/7411.png  
  inflating: face_age/face_age/024/7497.png  
  inflating: face_age/face_age/024/7511.png  
  inflating: face_age/face_age/024/7520.png  
  inflating: face_age/face_age/024/7558.png  
  inflating: face_age/face_age/024/7600.png  
  inflating: face_age/face_age/024/7629.png  
  inflating: face_age/face_age/024/765.png  
  inflating: face_age/face_age/024/7691.png  
  inflating: face_age/face_age/024/7771.png  
  inflating: face_age/face_age/024/7782.png  
  inflating: face_age/face_age/024/7817.png  
  inflating: face_age/face_age/024/786.png  
  inflating: face_age/face_age/024/7955.png  
  inflating: face_age/face_age/024/8099.png  
  inflating: face_age/face_age/

# Let's Start Here

In [ ]:
# Define train & val dir
train_dir = './face_age/face_age/'
val_dir = './face_age/face_age/'


def train_val_generator(TRAINING_DIR, VALIDATION_DIR):
  # Create imagegen
  train_datagen = ImageDataGenerator(rescale = 1./255.,
                                      rotation_range = 40,
                                      width_shift_range = 0.2,
                                      height_shift_range = 0.2,
                                      shear_range = 0.2,
                                      zoom_range = 0.2,
                                      horizontal_flip = True)

  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                        batch_size=32, 
                                                        class_mode='categorical',
                                                        target_size=(150, 150))

  test_datagen = ImageDataGenerator(rescale = 1./255.)

  validation_generator = test_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                        batch_size=32, 
                                                        class_mode='categorical',
                                                        target_size=(150, 150))
  return train_generator, validation_generator

train_generator, validation_generator = train_val_generator(train_dir, val_dir)

Found 9778 images belonging to 99 classes.
Found 9778 images belonging to 99 classes.


In [ ]:
print(train_generator, validation_generator)

<keras.preprocessing.image.DirectoryIterator object at 0x7f492282fe80> <keras.preprocessing.image.DirectoryIterator object at 0x7f4922142d10>


## Transfer Learning using  VGG16

In [ ]:
from keras.applications import VGG16

vgg = VGG16(weights='imagenet',  include_top=False, input_shape=(150, 150, 3))

# Freeze the layers of the VGG16 model
for layer in vgg.layers:
    layer.trainable = False

58889256/58889256 [==============================] - 2s 0us/step


## Implement Pre-Trained Model in CNN Model

In [ ]:
model = Sequential([      
      # VGG Layer
      vgg,
      Flatten(),
      Dropout(0.2),
      # 512 neuron hidden layer
      Dense(2048, activation='relu'),
      Dense(1024, activation='relu'),
      Flatten(),
      Dense(512, activation='relu'),
      Dense(99, activation='softmax')
  ])


In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 4, 4, 512)         14714688  
                                                                 
 flatten_8 (Flatten)         (None, 8192)              0         
                                                                 
 dropout_12 (Dropout)        (None, 8192)              0         
                                                                 
 dense_24 (Dense)            (None, 2048)              16779264  
                                                                 
 dense_25 (Dense)            (None, 1024)              2098176   
                                                                 
 flatten_9 (Flatten)         (None, 1024)              0         
                                                                 
 dense_26 (Dense)            (None, 512)              

## Set Callback if accuracy stagnant (does not improve) in 3 epochs

In [ ]:
callbacks = [    
    EarlyStopping(patience=3),    
]

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 100

model.fit(train_generator,
          epochs=epochs,          
          validation_data=validation_generator)

Epoch 1/100
306/306 [==============================] - 109s 349ms/step - loss: 3.7970 - accuracy: 0.1260 - val_loss: 3.3985 - val_accuracy: 0.1681
Epoch 2/100
306/306 [==============================] - 106s 348ms/step - loss: 3.4723 - accuracy: 0.1512 - val_loss: 3.2440 - val_accuracy: 0.1775
Epoch 3/100
306/306 [==============================] - 107s 351ms/step - loss: 3.3903 - accuracy: 0.1578 - val_loss: 3.2110 - val_accuracy: 0.1818
Epoch 4/100
306/306 [==============================] - 107s 350ms/step - loss: 3.3459 - accuracy: 0.1619 - val_loss: 3.1325 - val_accuracy: 0.1825
Epoch 5/100
306/306 [==============================] - 106s 347ms/step - loss: 3.3277 - accuracy: 0.1620 - val_loss: 3.1834 - val_accuracy: 0.1745
Epoch 6/100
306/306 [==============================] - 110s 358ms/step - loss: 3.2898 - accuracy: 0.1651 - val_loss: 3.1166 - val_accuracy: 0.1865
Epoch 7/100
306/306 [==============================] - 110s 361ms/step - loss: 3.2775 - accuracy: 0.1684 - val_loss: 3

In [ ]:
model.save('aging_classification_model.h5')

Load Model & Convert to TFlite

In [ ]:
model = tf.keras.models.load_model('/content/aging_classification_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("aging_classification_model.tflite", "wb").write(tflite_model)